In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#export
from exp.nb_01 import *

# refactor into a function from previous notebook 
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    #converts numpy into torch tensor    
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): 
    return (x-m)/s

In [6]:
x_train,y_train,x_valid,y_valid = get_data()

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [11]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [12]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-3.0466e-06), tensor(1.))

In [9]:
#export
def test_near_zero(a,tol=1e-3): 
    assert a.abs()<tol, f"Near zero: {a}"

In [13]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [15]:
# Getting a simple description of the data
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

# From basics

In [16]:
# number of hidden nodes
nh = 50

In [17]:
# kaiming init/he init
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)

w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [18]:
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [19]:
def lin(x, w, b):
    # Linear layer
    return x@w + b

In [20]:
t = lin(x_valid, w1, b1)

In [21]:
# if we have good init, then the result of the linear transform should still be well normalized
t.mean(),t.std()

(tensor(0.0302), tensor(0.9783))

In [22]:
def relu(x): 
    # simple activation
    return x.clamp_min(0.)


In [23]:
t = relu(lin(x_valid, w1, b1))

In [25]:
# Note, after relu the data is not well normalized 
t.mean(), t.std()

(tensor(0.4013), tensor(0.5913))

# alternative initialisation

In [26]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)
w1.mean(), w1.std()

(tensor(-4.6636e-05), tensor(0.0506))

In [28]:
# slightly better, but maybe not that much better
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5267), tensor(0.7861))

In [29]:
#export 
from torch.nn import init

In [30]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [32]:
t.mean(), t.std()

(tensor(0.4810), tensor(0.7853))

In [33]:
import torch.nn

In [34]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [35]:
torch.nn.Linear.forward??

In [36]:
torch.nn.functional.linear??


## try moving the bias?? 

In [37]:
def relu_b(x): return x.clamp_min(0.) - 0.5 

In [38]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu_b(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1380), tensor(0.9056))

# build a model

In [39]:
def model(xb):
    # using closures of weights
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [40]:
%timeit -n 10 _=model(x_valid)

10.2 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
# check target shapes line up
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

## Loss function

In [43]:
model(x_valid).shape

torch.Size([10000, 1])

In [53]:
#export

def mse(output, target):
    return (output.squeeze(-1) - target).pow(2).mean()

In [54]:
y_train.dtype

torch.float32

In [55]:
# convert to float for mse calc
y_train,y_valid = y_train.float(),y_valid.float()

In [56]:
preds = model(x_train)

In [57]:
mse(preds, y_train)

tensor(31.6283)

## make gradients and backward pass

In [62]:
# All these functions don't return a value, rather they mutate the input!!

def mse_grad(inp, targ):
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
    
def relu_grad(inp, out):
    inp.g  = (inp > 0).float() * out.g

def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)
    
def forward_and_backward(inp, targ):
    # Go forward:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    loss = mse(out,targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [63]:
forward_and_backward(x_train, y_train)

In [64]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [65]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [66]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [67]:
loss = forward(xt2, y_train)

In [68]:
loss.backward()

In [69]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Refactor... 

## OOP 4 evA, layers as classes

In [71]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        # Let's not use shifted relu like jeremy does... 
        self.out = inp.clamp_min(0.)
        
        return self.out
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [74]:
class Lin():
    def __init__(self, w, b):
        # Init needs to recieve parameters.
        self.w, self.b = w, b
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w +self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * 
                    self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [81]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    def backward(self):
        self.inp.g = 2.* (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [82]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [83]:
# clear gradients and make model
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [84]:
%time loss = model(x_train, y_train)

CPU times: user 496 ms, sys: 75.2 ms, total: 571 ms
Wall time: 71.5 ms


In [85]:
%time model.backward()

CPU times: user 7.12 s, sys: 6.03 s, total: 13.1 s
Wall time: 1.66 s


In [86]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## refactor Module.forward()

In [101]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [110]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [111]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [112]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [113]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [114]:
%time loss = model(x_train, y_train)

CPU times: user 521 ms, sys: 175 ms, total: 696 ms
Wall time: 87 ms


In [115]:
%time model.backward()

CPU times: user 755 ms, sys: 507 ms, total: 1.26 s
Wall time: 158 ms


In [116]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## without einsum

In [117]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [118]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [119]:
%time loss = model(x_train, y_train)

CPU times: user 553 ms, sys: 88.4 ms, total: 641 ms
Wall time: 80.2 ms


In [120]:
%time model.backward()

CPU times: user 831 ms, sys: 439 ms, total: 1.27 s
Wall time: 159 ms


In [121]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

# Now use pytorch Linear and Module

In [122]:
#export
from torch import nn

In [123]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [124]:
model = Model(m, nh, 1)

In [125]:
%time loss = model(x_train, y_train)

CPU times: user 489 ms, sys: 121 ms, total: 610 ms
Wall time: 76.4 ms
